(tonic-identification)=
# Tonic identification

```{note}
TODO: link to musicological sections.
```

Let us first start by installing and importing the latest released version of ``compiam``.

In [ ]:
%pip install compiam

## Importing compiam to the project
import compiam

# Import extras and supress warnings to keep the tutorial clean
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

```{note}
All notebooks in this tutorial are meant to be standalone. Therefore, for each tool walkthrough we will install and import `compiam`.
```

In [ ]:
pprint("Available methods for tonic identification: {}".\
    format(compiam.melody.tonic_identification.list_tools()))

In [ ]:
# Importing the tool
from compiam.melody.tonic_identification import TonicIndianMultiPitch

# We first initialize the tool we have just imported
tonic_multipitch = TonicIndianMultiPitch()

# Let's first see the specific attributes of this tool
attributes = [x for x in dir(tonic_multipitch) if "__" not in x]
pprint(attributes)

We observe a long list of attributes for this tool. That is because this is an **extractor**. Within the context of this tutorial, we use this concept to refer to **heuristic-based** tools that extract or compute a particular representation from a music signal. In this case, the tonic of the input musical recording. Heuristic-based approaches are commonly tuned by a list of particular parameter, which can be tuned to improve the performance for particular cases.

In [ ]:
# We can print out the standard value for a particular parameter
tonic_multipitch.minTonicFrequency

Let's now load the CompMusic Indian Art Music Tonic dataset to evaluate the performance of this approach.

In [ ]:
# Loading Tonic dataset using the mirdata loader
tonic_dataset = compiam.load_dataset("compmusic_indian_tonic", data_home="./workspace/")
tonic_dataset.download()
tonic_dataset.validate()


Right! We now have the dataset. Unfortunately, we observe that the audio tracks for this collections are not opnly available but only shared under explicit request. **However that is not a problem!** Possible solutions: 
* You may request the audios as follow the instructions on the dataloader to have them considered.
* Otherwise, if available, you may get the audios from the Dunya database.

Accessing the data in Dunya requires you to have a unique and non-shareable access token. For that reason, we cannot provide an interactive walkthrough of how to parse audio examples from Dunya.

**Not a problem though!** We list here an example code block that may be run to parse audio from Dunya, and we provide, within the tutorial materials, a couple of audio excerpts to show you through the available tools.

```{note}
You can get a random recording from the dataloader using ``dataset.choice_track()``. 
```

In [ ]:
random_track = tonic_dataset.choice_track()
print("MusicBrainz ID of random track: {}".format(random_track.mbid))

```python
import compiam
carnatic_corpora = compiam.load_corpora("carnatic", cc=True, token="<your-token-goes-here>")
print(carnatic_corpora.get_collection())  # Print out the available recordings in the database
print(carnatic_corpora.get_recording("<mbid>"))  # Print out available data for specific track
carnatic_corpora.download_mp3("<mbid>", "<path/to/save")  # Download and save mp3 audio for particular track
```

Let's select a couple of random recordings from the Indian Art Music Tonic dataset to extract the tonic from.